Primero importamos pandas y cargamos nuestro dataset desde una ubicación que pueda leerse con read_csv:

In [216]:
import pandas as pd

laptops_dataframe = pd.read_csv("https://raw.githubusercontent.com/rubengr16/MachineLearning/main/Prac1/train.csv", sep=",")
laptops_dataframe


,LaptopId,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,981,Toshiba,Portege Z30-C-1CV,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1195.00
1,996,HP,Spectre 13-V111dx,Ultrabook,13.3,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.11kg,1349.00
2,77,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,128GB SSD + 1TB HDD,Intel UHD Graphics 620,Windows 10,2.02kg,855.00
3,103,HP,Envy 13-ad009n,Ultrabook,13.3,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Nvidia GeForce MX150,Windows 10,1.38kg,1119.00
4,946,Dell,Latitude 7280,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics,Windows 10,1.36kg,1472.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,994,Dell,Inspiron 5378,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.68kg,889.00
908,615,Asus,FX753VD-GC007T (i7-7700HQ/8GB/1TB,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1050,Windows 10,3kg,1168.00
909,1063,Dell,Inspiron 5567,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,16GB,2TB HDD,AMD Radeon R7 M445,Windows 10,2.32kg,989.99
910,96,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R5 M430,Linux,2.2kg,599.90


In [217]:
def to_letter(size: list) -> list:
  unique_size = list(set(size))

  unique_size.sort()
  len(unique_size)
  gap_size = int(len(unique_size) / 3)
  dict_size = {unique_size[gap_size]: 'S', unique_size[gap_size * 2]: 'M'}
  dict_keys = list(dict_size.keys())

  for i in range(len(size)):
    if (size[i] < dict_keys[0]):
      size[i] = dict_size[dict_keys[0]]
    elif (size[i] <= dict_keys[1]):
      size[i] = dict_size[dict_keys[1]]
    else:
      size[i] = 'L'
  return (size)

In [218]:
def parse_inches(laptops_dataframe):
  inches = []

  for i in range(len(laptops_dataframe)):
    inches.append(int(laptops_dataframe.loc[i, 'Inches']))
  inches = to_letter(inches)
  laptops_dataframe['Inches'] = inches
  return (laptops_dataframe)


In [219]:
def get_size(screen_split: list) -> int:
  screen_split = str(screen_split[-1]).split("x")
  
  if (len(screen_split) != 2):
    # Si size no se representa por <num1>x<num2>, no comprendemos el valor
    # y ponemos un 0
    return (0)
  else:
    return (int(screen_split[0]) * int(screen_split[1]))


def get_resolution(name: str) -> str:
  resolution = ""
  name = name.lower()

  if ("full hd" in name or "fhd" in name):
    resolution = resolution + ' ' + "FHD"
  elif ("4k" in name or "ultra hd" in name or "uhd" in name):
    resolution = resolution + ' ' + "UHD"
  elif ("hd" in name):
    resolution = resolution + ' ' + "HD"
  if ("touchscreen" in name):
    resolution = resolution + ' ' + "TS"
  if ("ips" in name):
    resolution = resolution + ' ' + "IPS"
  return (resolution)



def parse_screen(laptops_dataframe):
  size = []

  for i in range(len(laptops_dataframe)):
    size.append(get_size(laptops_dataframe.loc[i,'ScreenResolution'].split()))
    laptops_dataframe.loc[i,'ScreenResolution'] = get_resolution(laptops_dataframe.loc[i, 'ScreenResolution'])
  to_letter(size)
  laptops_dataframe['Size'] = size
  return (laptops_dataframe)

In [220]:
def parse_ram(laptops_dataframe):
  for i in range(len(laptops_dataframe)):
    laptops_dataframe.loc[i,'Ram'] = int(laptops_dataframe.loc[i,'Ram'][:-2])
  return (laptops_dataframe)

In [221]:
def get_cpu_id(name: str) -> str:
  name = name.lower()

  if ("intel core" in name):
    return (name.split()[2])
  if ("intel celeron" in name):
    return ("celeron")
  if ("intel pentium" in name):
    return ("pentium")
  if ("amd" in name):
    return (name.split()[1].split('-')[0])

def parse_cpu(laptops_dataframe):
  speed = []

  for i in range(len(laptops_dataframe)):
    cpu_split = laptops_dataframe.loc[i,'Cpu'].split()
    speed.append(float(cpu_split[-1][:-3]))
    laptops_dataframe.loc[i,'Cpu'] = get_cpu_id(laptops_dataframe.loc[i,'Cpu'])
  speed = to_letter(speed)
  laptops_dataframe['Speed'] = speed
  return (laptops_dataframe)

In [222]:
def get_mem_type(name: str) -> str:
  mem_type = ""
  name = name.lower()

  if ("hdd" in name or "hybrid" in name):
    mem_type = mem_type + ' ' + "HDD"
  if ("ssd" in name or "hybrid" in name):
    mem_type = mem_type + ' ' + "SSD"
  return (mem_type)


def to_gb(num: int, unit: str) -> int:
  unit = unit.lower()
  if (unit == "gb"):
    return (num)
  else:
    return (num * 1024)


def get_total_size(size: str) -> str:
  size = size.split('+')
  size[0] = size[0].split()[0]
  num_size = to_gb(int(size[0][:-2].split('.')[0]), size[0][-2:])
  if (len(size) > 1):
    size[1] = size[1].split()[0]
    num_size += to_gb(int(size[1][:-2].split('.')[0]), size[0][-2:])
  return (num_size)

def parse_memory(laptops_dataframe):
  mem_type = []

  for i in range(len(laptops_dataframe)):
    mem_type.append(get_mem_type(laptops_dataframe.loc[i,'Memory']))
    laptops_dataframe.loc[i,'Memory'] = get_total_size(laptops_dataframe.loc[i,'Memory'])
  laptops_dataframe['MemoryType'] = mem_type
  
  return (laptops_dataframe)

In [223]:
def parse_gpu(laptops_dataframe):
  for i in range(len(laptops_dataframe)):
    laptops_dataframe.loc[i,'Gpu'] = laptops_dataframe.loc[i,'Gpu'].split()[0]
  return (laptops_dataframe)

In [224]:
def parse_weight(laptops_dataframe):
  weight = []

  for i in range(len(laptops_dataframe)):
    weight.append(float(laptops_dataframe.loc[i, 'Weight'][:-2]))
  weight = to_letter(weight)
  laptops_dataframe['Weight'] = weight
  return (laptops_dataframe)

Parseo principal y preprocesado de datos, mediante las funciones creadas anteriormente

In [225]:
laptops_dataframe = parse_inches(laptops_dataframe)
laptops_dataframe = parse_screen(laptops_dataframe)
laptops_dataframe = parse_ram(laptops_dataframe)
laptops_dataframe = parse_cpu(laptops_dataframe)
laptops_dataframe = parse_memory(laptops_dataframe)
laptops_dataframe = parse_gpu(laptops_dataframe)
laptops_dataframe = parse_weight(laptops_dataframe)

laptops_dataframe

,LaptopId,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price,Size,Speed,MemoryType
0,981,Toshiba,Portege Z30-C-1CV,Notebook,M,FHD,i5,4,128,Intel,Windows 7,S,1195.00,S,M,SSD
1,996,HP,Spectre 13-V111dx,Ultrabook,M,FHD TS IPS,i7,8,256,Intel,Windows 10,S,1349.00,S,L,SSD
2,77,Dell,Inspiron 5570,Notebook,L,FHD,i7,8,129,Intel,Windows 10,M,855.00,S,M,HDD SSD
3,103,HP,Envy 13-ad009n,Ultrabook,M,FHD IPS,i7,8,256,Nvidia,Windows 10,S,1119.00,S,L,SSD
4,946,Dell,Latitude 7280,Ultrabook,M,FHD TS,i5,8,256,Intel,Windows 10,S,1472.20,S,M,SSD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,994,Dell,Inspiron 5378,2 in 1 Convertible,M,FHD TS,i5,8,256,Intel,Windows 10,M,889.00,S,M,SSD
908,615,Asus,FX753VD-GC007T (i7-7700HQ/8GB/1TB,Gaming,L,FHD,i7,8,129,Nvidia,Windows 10,L,1168.00,S,L,HDD SSD
909,1063,Dell,Inspiron 5567,Notebook,L,FHD,i7,16,2048,AMD,Windows 10,M,989.99,S,L,HDD
910,96,Dell,Inspiron 3567,Notebook,L,FHD,i7,8,1024,AMD,Linux,M,599.90,S,L,HDD


Aplicamos el preprocesado de datos para transformar toda la información obtenida y convertida en el paso anterior

In [ ]:
import numpy as np
import sklearn.compose

np_laptops = np.array(laptops_dataframe)
column_transformer = sklearn.compose.ColumnTransformer(transformers=[
    ("id", "drop", [0]),
    ("Company", sklearn.preprocessing.OrdinalEncoder(), [1]),
    ("product", "drop", [2]),
    ("TypeName", sklearn.preprocessing.OrdinalEncoder(), [3]),
    ("Inches", sklearn.preprocessing.StandardScaler(), [4]),
    ("ScreenResolution", sklearn.preprocessing.OrdinalEncoder(), [5]),
    ("Cpu", sklearn.preprocessing.StandardScaler(), [6]),
    ("Ram", sklearn.preprocessing.MinMaxScaler(), [7]),
    ("Memory", sklearn.preprocessing.OrdinalEncoder(), [8]),
    ("Gpu", sklearn.preprocessing.OrdinalEncoder(), [9]),
    ("OpSys", sklearn.preprocessing.OrdinalEncoder(), [10]),
    ("Weight", sklearn.preprocessing.StandardScaler(), [11]),
    ("price", "drop", [12])
]);
data = column_transformer.fit_transform(np_laptops)

column_transformer = sklearn.compose.ColumnTransformer(transformers=[
    ("id", "drop", [0]),
    ("company", "drop", [1]),
    ("product", "drop", [2]),
    ("typeName", "drop", [3]),
    ("inches", "drop", [4]),
    ("screenResolution", "drop", [5]),
    ("cpu", "drop", [6]),
    ("ram", "drop", [7]),
    ("memory", "drop", [8]),
    ("gpu", "drop", [9]),
    ("opSys", "drop", [10]),
    ("weight", "drop", [11]),
    ("Price", "passthrough", [12])
]);
target = column_transformer.fit_transform(np_laptops)

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(data, target)

In [ ]:
reg.score(data, target)

0.7078667385062551

In [ ]:
test_dataframe = pd.read_csv("https://raw.githubusercontent.com/rubengr16/MachineLearning/main/Prac1/test.csv", sep=",")

# Obtener sólo la resolución en píxeles
for i in range(len(test_dataframe)):
  screen_split = test_dataframe.loc[i,'ScreenResolution'].split()
  test_dataframe.loc[i,'ScreenResolution'] = screen_split[-1:]

# Obtener los GHz del procesador
for i in range(len(test_dataframe)):
  cpu_split = test_dataframe.loc[i,'Cpu'].split()
  cpu_str = str(cpu_split[-1:])
  test_dataframe.loc[i,'Cpu'] = cpu_str[2:-5]

# Eliminar el GB
for i in range(len(test_dataframe)):
  test_dataframe.loc[i,'Ram'] = float(test_dataframe.loc[i,'Ram'][:-2])

# Eliminar el kg
for i in range(len(test_dataframe)):
  test_dataframe.loc[i,'Weight'] = float(test_dataframe.loc[i,'Weight'][:-2])

np_test = np.array(test_dataframe)

column_transformer = sklearn.compose.ColumnTransformer(transformers=[
    ("id", "drop", [0]),
    ("Company", sklearn.preprocessing.OrdinalEncoder(), [1]),
    ("product", "drop", [2]),
    ("TypeName", sklearn.preprocessing.OrdinalEncoder(), [3]),
    ("Inches", sklearn.preprocessing.StandardScaler(), [4]),
    ("ScreenResolution", sklearn.preprocessing.OrdinalEncoder(), [5]),
    ("Cpu", sklearn.preprocessing.StandardScaler(), [6]),
    ("Ram", sklearn.preprocessing.MinMaxScaler(), [7]),
    ("Memory", sklearn.preprocessing.OrdinalEncoder(), [8]),
    ("Gpu", sklearn.preprocessing.OrdinalEncoder(), [9]),
    ("OpSys", sklearn.preprocessing.OrdinalEncoder(), [10]),
    ("Weight", sklearn.preprocessing.StandardScaler(), [11])
]);
data = column_transformer.fit_transform(np_test)



array([[ 1.48178283e+03],
       [ 8.14249312e+02],
       [ 9.27590421e+02],
       [ 7.22447346e+02],
       [ 7.87414914e+02],
       [ 1.69552895e+02],
       [ 7.54213358e+02],
       [ 6.74577196e+02],
       [ 6.99292458e+02],
       [ 8.99412655e+02],
       [ 3.10724797e+02],
       [ 1.21916706e+03],
       [ 3.33863468e+02],
       [ 1.02660608e+03],
       [ 4.93116372e+02],
       [ 1.21331111e+03],
       [ 1.20507114e+03],
       [ 9.28322733e+02],
       [ 1.01123041e+02],
       [ 6.24462354e+02],
       [ 1.01350947e+03],
       [ 1.19586415e+03],
       [ 2.90243012e+02],
       [ 7.91505873e+02],
       [ 2.75384426e+02],
       [ 5.77263855e+02],
       [ 5.37202230e+02],
       [ 1.18295199e+03],
       [ 9.76657336e+02],
       [ 5.21247981e+02],
       [ 3.64331143e+02],
       [ 5.96668126e+02],
       [ 7.46150679e+02],
       [ 7.51869292e+02],
       [ 6.93919435e+02],
       [ 8.34337506e+02],
       [ 1.03364470e+03],
       [ 4.92657485e+02],
       [ 8.9

In [ ]:
prediction = reg.predict(data)
prediction_dataframe = pd.DataFrame(prediction)
ids = test_dataframe.iloc[:,0]
submit = pd.concat([ids,prediction_dataframe], axis=1)
submit.columns=['LaptopId','Price']
submit.to_csv("submitLinearRegresion.csv", index=False)